In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [2]:
#  Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame
weather_csv = "resources/weather_data.csv"
weather_data_df = pd.read_csv(weather_csv)
weather_df = pd.DataFrame(weather_data_df)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tarakan,3.30,117.63,77.04,82,100,10.38,ID,1596243492
1,1,Tasiilaq,65.61,-37.64,44.60,65,0,5.55,GL,1596242972
2,2,Puerto El Triunfo,13.28,-88.55,86.00,79,20,3.36,SV,1596243493
3,3,Cabo San Lucas,22.89,-109.91,88.00,54,20,10.29,MX,1596242732
4,4,Rawlins,41.79,-107.24,84.20,16,1,6.93,US,1596243493


In [3]:
# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.
gmaps.configure(api_key=g_key)
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.

# referenced https://stackoverflow.com/questions/31617845/how-to-select-rows-in-a-dataframe-between-two-values-in-python-pandas

temp_weather_df = weather_df[weather_df["Max Temp"].between(75, 85)] 
cloud_weather_df = temp_weather_df[temp_weather_df["Cloudiness"].between(0, 10)]
vacay_weather_df=cloud_weather_df[cloud_weather_df["Wind Speed"].between(0, 5)]
vacay_weather_df
# Drop any rows will null values.


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,Bethel,41.37,-73.41,77.00,69,1,3.36,US,1596242920
20,20,San Javier,37.81,-0.84,81.00,100,0,4.70,ES,1596243498
52,52,Beyneu,45.32,55.20,76.33,26,7,4.72,KZ,1596243506
75,75,Bolobo,-2.17,16.23,76.60,59,0,3.22,CD,1596243047
99,99,Albany,42.60,-73.97,79.00,56,0,3.80,US,1596243517
139,139,Avera,33.19,-82.53,82.40,78,1,3.36,US,1596243527
183,183,El Ejido,36.78,-2.81,80.60,88,0,4.70,ES,1596243538
473,473,Fethiye,36.62,29.12,77.00,78,0,3.36,TR,1596243611
476,476,Fairbanks,64.84,-147.72,77.00,38,1,3.98,US,1596243568
544,544,Jacareacanga,-6.22,-57.75,75.11,49,0,0.63,BR,1596243633


In [5]:
# add empty lists for hotel info to be added to data frame
vacay_weather_df["hotel_name"] = ""
vacay_weather_df["hotel_address"] = ""
vacay_weather_df["hotel_lat"] = ""
vacay_weather_df["hotel_lng"] = ""
vacay_weather_df.head()

/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value 

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name,hotel_address,hotel_lat,hotel_lng
16,16,Bethel,41.37,-73.41,77.00,69,1,3.36,US,1596242920,,,,
20,20,San Javier,37.81,-0.84,81.00,100,0,4.70,ES,1596243498,,,,
52,52,Beyneu,45.32,55.20,76.33,26,7,4.72,KZ,1596243506,,,,
75,75,Bolobo,-2.17,16.23,76.60,59,0,3.22,CD,1596243047,,,,
99,99,Albany,42.60,-73.97,79.00,56,0,3.80,US,1596243517,,,,


In [6]:
# set up a parameters dictionary
params ={
        "radius": 5000,
         "types": "lodging",
         "key": g_key
}
# Use the lat/lng we recovered to identify hotels
for index, row in vacay_weather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.

    try:
        vacay_weather_df.loc[index, "hotel_name"] = name_address["results"][0]["name"]
        vacay_weather_df.loc[index, "hotel_address"] = name_address["results"][0]["vicinity"]
        vacay_weather_df.loc[index, "hotel_lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        vacay_weather_df.loc[index, "hotel_lng"] = name_address["results"][0]["geometry"]["location"]["lng"]

    except:
        pass
vacay_weather_df

/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name,hotel_address,hotel_lat,hotel_lng
16,16,Bethel,41.37,-73.41,77.00,69,1,3.36,US,1596242920,Courtyard by Marriott Danbury,"3 Eagle Road, Danbury",41.4114,-73.4153
20,20,San Javier,37.81,-0.84,81.00,100,0,4.70,ES,1596243498,Hotel Mar Menor,"Avenida Mar Menor, 4, Santiago de la Ribera",37.7991,-0.809663
52,52,Beyneu,45.32,55.20,76.33,26,7,4.72,KZ,1596243506,NUR,Beyneu,45.321,55.1924
75,75,Bolobo,-2.17,16.23,76.60,59,0,3.22,CD,1596243047,,,,
99,99,Albany,42.60,-73.97,79.00,56,0,3.80,US,1596243517,,,,
139,139,Avera,33.19,-82.53,82.40,78,1,3.36,US,1596243527,,,,
183,183,El Ejido,36.78,-2.81,80.60,88,0,4.70,ES,1596243538,Gran Hotel Victoria,"Bulevar de el Ejido, 400, El Ejido",36.7739,-2.80889
473,473,Fethiye,36.62,29.12,77.00,78,0,3.36,TR,1596243611,Club & Hotel Letoonia,"Karagözler, Fevzi Çakmak Caddesi No:82",36.6379,29.0983
476,476,Fairbanks,64.84,-147.72,77.00,38,1,3.98,US,1596243568,Westmark Fairbanks Hotel & Conference Center,"813 Noble Street, Fairbanks",64.8399,-147.712
544,544,Jacareacanga,-6.22,-57.75,75.11,49,0,0.63,BR,1596243633,Casa do Ian😎,"Travessa Tenente Fernandes, Jacareacanga",-6.2237,-57.756


In [14]:

vacay_weather_df["hotel_lat"] = pd.to_numeric(vacay_weather_df["hotel_lat"], downcast="float")
vacay_weather_df["hotel_lng"] = pd.to_numeric(vacay_weather_df["hotel_lng"], downcast="float")
vacay_weather_df.dropna(subset = ['hotel_lat'], inplace=True)
vacay_weather_df

/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/erinhislope/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name,hotel_address,hotel_lat,hotel_lng
16,16,Bethel,41.37,-73.41,77.00,69,1,3.36,US,1596242920,Courtyard by Marriott Danbury,"3 Eagle Road, Danbury",41.411369,-73.415268
20,20,San Javier,37.81,-0.84,81.00,100,0,4.70,ES,1596243498,Hotel Mar Menor,"Avenida Mar Menor, 4, Santiago de la Ribera",37.799080,-0.809663
52,52,Beyneu,45.32,55.20,76.33,26,7,4.72,KZ,1596243506,NUR,Beyneu,45.320980,55.192360
183,183,El Ejido,36.78,-2.81,80.60,88,0,4.70,ES,1596243538,Gran Hotel Victoria,"Bulevar de el Ejido, 400, El Ejido",36.773888,-2.808889
473,473,Fethiye,36.62,29.12,77.00,78,0,3.36,TR,1596243611,Club & Hotel Letoonia,"Karagözler, Fevzi Çakmak Caddesi No:82",36.637909,29.098297
476,476,Fairbanks,64.84,-147.72,77.00,38,1,3.98,US,1596243568,Westmark Fairbanks Hotel & Conference Center,"813 Noble Street, Fairbanks",64.839905,-147.712341
544,544,Jacareacanga,-6.22,-57.75,75.11,49,0,0.63,BR,1596243633,Casa do Ian😎,"Travessa Tenente Fernandes, Jacareacanga",-6.223699,-57.756012
546,546,Senador José Porfírio,-2.59,-51.95,78.94,65,0,2.39,BR,1596243634,MORADA DO SOL pousada,"Tr. São Francisco PROXIMO AO RIO XINGÚ, Senado...",-2.592242,-51.950932
561,561,Wuwei,37.93,102.64,79.00,43,0,3.85,CN,1596243638,Thank Inn Chain Hotel gansu wuwei liangzhou di...,"5-1 西关中路, Liangzhou District, Wuwei",37.922981,102.615395


In [19]:
# Plot markers on top of the heatmap
locations = vacay_weather_df[["hotel_lat", "hotel_lng"]]
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:

#  # NOTE: Do not change any of the code in this cell
# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{hotel_name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # # Store the DataFrame Row

# # # NOTE: be sure to update with your DataFrame name
# vacay_weather_df = [info_box_template.format(**row) for index, row in vacay_weather_df.iterrows()]
# locations = vacay_weather_df[["hotel_lat", "hotel_lng"]]

AttributeError: 'list' object has no attribute 'iterrows'